In [12]:
import emcee
import matplotlib
import numpy as np
import corner
import scipy.optimize as op
import timeit

%matplotlib inline  

# some important variables
g = 6.6743e-8
msun = 1.989e33
au = 1.496e13
pi = 3.14159

In [2]:
nbin = 58 ## number of binaries in sample
nvar = 5

## read in delK, parallax
delk = np.zeros(nbin)
edelk = np.zeros(nbin)
plxval = np.zeros(nbin)
plxprior = np.zeros(nbin)
name = strs = ['']*nbin
k = np.zeros(nbin)
ek = np.zeros(nbin)
feh = np.zeros(nbin)
f = open('data2.txt','r')
header1 = f.readline()
i=0
for line in f:
    line = line.strip()
    columns = line.split()
    name[i] = columns[0]
    delk[i] = float(columns[1])
    edelk[i] = float(columns[2])
    plxval[i] = float(columns[3])
    plxprior[i] = float(columns[4])
    k[i] = float(columns[5])
    ek[i] = float(columns[6])
    feh[i] = float(columns[8])
    i+=1
f.close()

## now for the sma**3/per**2
f = open('fits.txt','r')
header1 = f.readline()
i=0
smaper = np.zeros(nbin)
esmaper = np.zeros(nbin)
for line in f:
    line = line.strip()
    columns = line.split()
    smaper[i] = float(columns[0])
    esmaper[i] = float(columns[1])
    i+=1
f.close()

fluxratio = 10.0**(delk/2.5)
del_eps = 2.5*np.log10(1.0+1.0/fluxratio)
kp = del_eps+k
ks = kp + delk

# compute MC errors on Kp, Ks
mcsize = 50000
ekp = kp*0.
eks = ks*0.
for i in range(0,len(ks)):
    ktmp = k[i]+ek[i]*np.random.standard_normal(mcsize)
    deltmp = delk[i]+edelk[i]*np.random.standard_normal(mcsize)
    fluxratio = 10.0**(deltmp/2.5)
    del_eps = 2.5*np.log10(1.0+1.0/fluxratio)
    kpt = del_eps+ktmp
    kst = kp[i] + ktmp
    ekp[i] = np.std(kpt)
    eks[i] = np.std(kst)


result_ben = np.array([0.2311,-0.1352, 0.0400, 0.0038, -0.0032]) # benedict fit value
result1 = np.array([0.23323026,-0.10887911, 0.019990399, 0.00027286744, -0.00046073982])# Mann fit value
result2 = plxval
result_delf = [0.001*1.8,0.001*6.12,0.001*13.205,-6.2315*0.001,0.001*0.37529]
result3= np.array([-0.663541,-0.212175 , 0.00594868 ,  0.00641549, -0.000416446]) ## one I measured in IDL
result3= np.array([-0.64329003970411125,-0.20043764801036046,-0.0041776516821012992,0.0033655838569557478,0.00030012649616073089])
fehcoeff = np.array([0.01])
result = np.concatenate([result3[0:4],fehcoeff,result2])

In [3]:
factor = (au**3.)*((4.0*np.pi**2.)/(g*msun))
empmass = factor*smaper/plxval**3
e_empmass = empmass*np.sqrt((esmaper/smaper)**2 +9.0*(plxprior/plxval)**2)

In [85]:
def lnlike(theta, smaper, esmaper, kp, ks, ekp, eks, feh, nvar):
    zp = 7.5e0
    au = 1.496e13
    msun = 1.989e33
    g = 6.6743e-8 
    a, b, c, d, f = theta[0:nvar]
    mplx = theta[nvar:theta.size]
    if np.min(mplx) <= 0:
        return -np.inf
    factor = (au**3.)*((4.0*np.pi**2.)/(g*msun))
    empmass = factor*smaper/mplx**3
    e_empmass = empmass*(esmaper/smaper)**2
    mka = kp - 5.0*(np.log10(1000.0/mplx)-1.) - zp
    mkb = ks - 5.0*(np.log10(1000.0/mplx)-1.) - zp
    mass1 = (10.0**(a + b*mka + c*mka**2. + d*mka**3.))*(1+f*feh)
    mass2 = (10.0**(a + b*mkb + c*mkb**2. + d*mkb**3.))*(1+f*feh)
    if np.min(mass1) <= 0 or np.min(mass2) <= 0:
        return -np.inf
    
    ## this is where we check to see if the relation always does brighter=higher mass (if not return -np.inf)
    #rng = [np.min(np.concatenate([mka,mkb])),np.max(np.concatenate([mka,mkb]))]
    mk = np.linspace(4.2,11.1,100)
    #l = 10.0**(a + b*(mk-zp) + c*(mk-zp)**2. + d*(mk-zp)**3.)
    #if np.max(np.gradient(l,mk)) > 0:
    #    return -np.inf
    check = all(l[i] >= l[i+1] for i in xrange(len(l)-1))
    if not check:
        return -np.inf

    mka_err = ekp
    mkb_err = eks
    mass1_err = np.abs((np.log(10.)*(b+2.*c*mka+3.*d*mka**2))*mass1*mka_err)
    mass2_err = np.abs((np.log(10.)*(b+2.*c*mkb+3.*d*mkb**2))*mass2*mkb_err)
    
    model_err = np.sqrt(mass1_err**2+mass2_err**2)
    model = mass1+mass2
    inv_sigma2 = 1.0/np.sqrt(e_empmass**2+model_err**2)
    return -0.5*(np.sum((empmass-model)**2*inv_sigma2 - np.log(inv_sigma2)))

In [86]:
def lnprior(theta, plxval, plxprior, nvar):
    mplx = theta[nvar:theta.size]
    lp = 0
    if np.min(mplx) <= 0:
        return -np.inf
    #for i in range(0,len(mplx)):
    #    lp += ((np.float(mplx[i])-np.float(plxval[i]))**2)/(np.float(plxprior[i])**2)
    lp += np.sum(((mplx-plxval)**2)/(plxprior)**2)
    lp*=(-0.5)
    if not np.isfinite(lp):
        return 0.0
    return lp

In [87]:
def lnprob(theta, plxval, plxprior, smaper, esmaper, kp, ks, ekp, eks, feh, nvar):
    lp = lnprior(theta, plxval, plxprior, nvar)
    if not np.isfinite(lp):
        return -np.inf
    like = lnlike(theta, smaper, esmaper, kp, ks, ekp, eks, feh, nvar)
    if not np.isfinite(like):
        return -np.inf
    val = lp + like
    return val

In [88]:
mplx = plxval + 1
lp = 0
for i in range(0,len(mplx)):
        lp += ((np.float(mplx[i])-np.float(plxval[i]))**2)/(np.float(plxprior[i])**2)
print lp
print np.sum(((mplx-plxval)**2)/(plxprior)**2)

193.071414774
193.071414774


In [89]:
a,b,c,d = result3[0:4]
zp = 7.5
mk = np.linspace(4.2,11.1,100)
l = 10.0**(a + b*(mk-zp) + c*(mk-zp)**2. + d*(mk-zp)**3.)
check = all(l[i] >= l[i+1] for i in xrange(len(l)-1))
#print l
#print np.max(np.gradient(l,mk))

In [90]:
import time
start_time = time.time()
for i in range(10000):
    lnprob(result,plxval,plxprior,smaper,esmaper,kp,ks,ekp,eks,feh,nvar)
end_time = time.time()
print end_time-start_time

2.57935786247
